In [3]:
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import pickle


In [4]:
## Load models and pickle files
model = load_model('model.h5')

with open('label_encoder_gender.pkl', 'rb') as f:
    label_encoder_gender = pickle.load(f)

with open('one_hot_encoder_geography.pkl', 'rb') as f:
    one_hot_encoder_geography = pickle.load(f)

with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

In [5]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [6]:
# Convert the input_data to a DataFrame
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [7]:
## Convert the gender into a numerical value

input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [13]:
## Convert the Geography into one-hot encoded values
geography_encoded = one_hot_encoder_geography.transform(input_df[['Geography']]).toarray()
geography_encoded = pd.DataFrame(geography_encoded, columns=one_hot_encoder_geography.get_feature_names_out(['Geography']))
geography_encoded

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [ ]:
## Drop the original Geography column and concatenate the one-hot encoded columns
input_df = input_df.drop(columns=['Geography'], axis=1)
input_df = pd.concat([input_df, geography_encoded], axis=1)

In [19]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [20]:
# Scale the input data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [21]:
# Predict the probability of churn
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 619ms/step


array([[0.02349714]], dtype=float32)

In [23]:
prediction_probability = prediction[0][0]
prediction_probability

np.float32(0.023497136)

In [24]:
if prediction_probability > 0.5:
    print("The customer is likely to churn with a probability of {:.2f}%".format(prediction_probability * 100))
else:
    print("The customer is unlikely to churn with a probability of {:.2f}%".format((1 - prediction_probability) * 100))

The customer is unlikely to churn with a probability of 97.65%
